In [ ]:
# jupyter serverextension enable --py jupyter_http_over_ws
# jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
import matplotlib as plt

os.chdir('C:/Users/Garrett/Downloads/backtest_data')

tickers = pd.read_csv('sp100.csv', names = ['ticker'], squeeze = True)

tickers = tickers + '_5min.txt'

In [4]:
# get SPY
header_list = ['datetime', 'open', 'high', 'low', 'close', 'volume']
spy_df = pd.read_csv('SPY_5min.txt', names = header_list, index_col = 'datetime', parse_dates=True)

# remove after-hours trading (before 9:30 AM and after 3:55 PM bars)
spy_df_rth = spy_df.between_time('09:30:00', '15:55:00')

# get daily closes
spy_daily = spy_df.between_time('15:55:00', '15:55:00')['close']

# get a vector of prior day closing prices
spy_prior_close = spy_daily.reset_index()
spy_prior_close['date'] = spy_prior_close['datetime'].dt.date
spy_prior_close['date'] = spy_prior_close['date'].shift(-1)
spy_prior_close.rename({'close': 'prior_close'}, axis=1, inplace=True)
spy_prior_close.drop(columns=['datetime'], inplace=True)
spy_prior_close.drop(spy_prior_close.tail(1).index,inplace=True)
spy_prior_close = spy_prior_close[['date', 'prior_close']]

# map the closing prices vector to each SPY 5m bar by date
spy_df_datetime = spy_df_rth.reset_index()
spy_df_datetime['date'] = spy_df_datetime['datetime'].dt.date
spy_df_datetime = spy_df_datetime.merge(spy_prior_close, how='left', on='date')

# subtract the prior day vector from the 5m closing price vector
spy_df_datetime['spy_up_down'] = spy_df_datetime['close'] - spy_df_datetime['prior_close']

# SPY ATR over last hour (twelve 5-min bars)
spy_df_rth = spy_df_rth.reset_index()
high_low = spy_df_rth['high'] - spy_df_rth['low']
high_cp = np.abs(spy_df_rth['high'] - spy_df_rth['close'].shift())
low_cp = np.abs(spy_df_rth['low'] - spy_df_rth['close'].shift())
tr_df = pd.concat([high_low, high_cp, low_cp], axis=1)
spy_true_range = np.max(tr_df, axis=1)
spy_df_rth['spy_atr'] = spy_true_range.rolling(12).mean()
spy_df_rth['power_index'] = (spy_df_rth['close'] - spy_df_rth['close'].shift(12)) / spy_df_rth['spy_atr']

In [5]:
def vwap(df):
    h = df.high.values
    l = df.low.values
    c = df.close.values
    v = df.volume.values
    return df.assign(vwap=((h+l+c)/3 * v).cumsum() / v.cumsum())

In [ ]:
# loop

os.chdir('C:/Users/Garrett/Downloads/backtest_data/tickers')
all_trades_df = pd.DataFrame(columns = ['Long/Short', 'Entry Datetime', 'Entry Price', 'Exit Datetime', 'Exit Price', 'PnL', 'Ticker'])

# for ticker_file in tickers.head(1):
for ticker_file in tickers:
  print(ticker_file)
  ticker_df = pd.read_csv(ticker_file, names = header_list, index_col = 'datetime', parse_dates=True)
  ticker_df_rth = ticker_df.between_time('09:30:00', '15:55:00')

  ticker_daily = ticker_df.between_time('15:55:00', '15:55:00')

  pd.options.mode.chained_assignment = None
  ticker_daily['sma20'] = ticker_daily['close'].rolling(window = 20, min_periods = 1).mean()
  ticker_daily['sma50'] = ticker_daily['close'].rolling(window = 50, min_periods = 1).mean()
  ticker_daily['sma100'] = ticker_daily['close'].rolling(window = 100, min_periods = 1).mean()
  ticker_daily['sma200'] = ticker_daily['close'].rolling(window = 200, min_periods = 1).mean()
  pd.options.mode.chained_assignment = 'warn'

  ticker_daily = ticker_daily.reset_index()
  ticker_daily['date'] = ticker_daily['datetime'].dt.date
  ticker_daily['date'] = ticker_daily['date'].shift(-1)

  # map the SMAs to the intraday prices
  ticker_df_rth = ticker_df_rth.reset_index()
  ticker_df_rth['date'] = ticker_df_rth['datetime'].dt.date

  ticker_df_rth = ticker_df_rth.merge(ticker_daily[['date', 'sma20', 'sma50',	'sma100',	'sma200']], how='left', on='date')

  # map the SPY up/down indicator to the 5m bars for the ticker
  ticker_df_rth = ticker_df_rth.merge(spy_df_datetime[['datetime', 'spy_up_down']], how='left', on='datetime')

  # calculate VWAP for ticker
  ticker_df_rth.index = pd.to_datetime(ticker_df_rth.date)
  ticker_df_rth = ticker_df_rth.groupby(ticker_df_rth.index.date, group_keys=False).apply(vwap)
  ticker_df_rth['vwap'] = ticker_df_rth['vwap'].shift(1)

  # ticker ATR over last hour
  high_low = ticker_df_rth['high'] - ticker_df_rth['low']
  high_cp = np.abs(ticker_df_rth['high'] - ticker_df_rth['close'].shift())
  low_cp = np.abs(ticker_df_rth['low'] - ticker_df_rth['close'].shift())

  tr_df = pd.concat([high_low, high_cp, low_cp], axis=1)
  ticker_true_range = np.max(tr_df, axis=1)

  ticker_df_rth['atr'] = ticker_true_range.rolling(12).mean()

  # map SPY power index to ticker
  ticker_df_rth = ticker_df_rth.merge(spy_df_rth[['datetime', 'power_index']], how='left', on='datetime')

  # calculate relative strengh/weakness for ticker
  ticker_df_rth['rsrw'] = (ticker_df_rth['close'] - ticker_df_rth['close'].shift(11)) / ticker_df_rth['atr'] - ticker_df_rth['power_index']

  # cut out first 200 days to start with enough data for all SMAs
  ticker_dates = ticker_df_rth['date'].unique()
  ticker_df_rth = ticker_df_rth[ticker_df_rth['date'] >= ticker_dates[200]]

  in_trade = 0
  trades_df = pd.DataFrame(columns = ['Long/Short', 'Entry Datetime', 'Entry Price', 'Exit Datetime', 'Exit Price'])

  for index, row in ticker_df_rth.iterrows():
    # entries
    if in_trade == 0 and row['datetime'].time() >= datetime.time(10, 30):
      # if SPY is up and ticker is above all SMAs, VWAP, and has relative strength, go long
      if row['spy_up_down'] > 0 and row['close'] > row['sma20'] and row['close'] > row['sma50'] and row['close'] > row['sma100'] and row['close'] > row['sma200'] and row['close'] > row['vwap'] and row['rsrw'] > 0:
        in_trade = 1
        entry_info = pd.DataFrame({'Long/Short': ['Long'], 'Entry Datetime': [ticker_df_rth['datetime'][index+1]], 'Entry Price': [ticker_df_rth['close'][index+1]]})
        trades_df = pd.concat([trades_df, entry_info], ignore_index = True)
      # if SPY is down and ticker is below all SMAs, VWAP, and has relative weakness, go short
      elif row['spy_up_down'] < 0 and row['close'] < row['sma20'] and row['close'] < row['sma50'] and row['close'] < row['sma100'] and row['close'] < row['sma200'] and row['close'] < row['vwap'] and row['rsrw'] < 0:
        in_trade = 1
        entry_info = pd.DataFrame({'Long/Short': ['Short'], 'Entry Datetime': [ticker_df_rth['datetime'][index+1]], 'Entry Price': [ticker_df_rth['close'][index+1]]})
        trades_df = pd.concat([trades_df, entry_info], ignore_index = True)
    # exits
    if in_trade == 1:
      # if at least 4 of: long and SPY goes negative, ticker goes below VWAP or any SMA, or loses relative strength, close long
      if trades_df.tail(1)['Long/Short'].item() == 'Long':
        if (row['spy_up_down'] < 0) + (row['close'] < row['sma20']) +  \
        (row['close'] < row['sma50']) + (row['close'] < row['sma100']) + \
        (row['close'] < row['sma200']) + (row['close'] < row['vwap']) + (row['rsrw'] < 0) > 3:
          in_trade = 0
          trades_df.loc[trades_df.index[-1], 'Exit Datetime'] = ticker_df_rth['datetime'][index+1]
          trades_df.loc[trades_df.index[-1], 'Exit Price'] = ticker_df_rth['close'][index+1]
      # if at least 4 of: short and SPY goes positive, ticker goes above VWAP or any SMA, or loses relative weakness, close short
      else:
        if (row['spy_up_down'] > 0) + (row['close'] > row['sma20']) + \
      (row['close'] > row['sma50']) + (row['close'] > row['sma100']) + \
      (row['close'] > row['sma200']) + (row['close'] > row['vwap']) + (row['rsrw'] > 0) > 3:
          in_trade = 0
          trades_df.loc[trades_df.index[-1], 'Exit Datetime'] = ticker_df_rth['datetime'][index+1]
          trades_df.loc[trades_df.index[-1], 'Exit Price'] = ticker_df_rth['close'][index+1]

  # get PnL of trades
  trades_df['PnL'] = trades_df['Exit Price'] / trades_df['Entry Price'] - 1

  # reverse sign for shorts
  trades_df.loc[trades_df['Long/Short'] == 'Short', 'PnL'] = trades_df.loc[trades_df['Long/Short'] == 'Short', 'PnL'] * -1

  # if last trade is still open remove it
  if np.isnan(trades_df.loc[trades_df.index[-1], 'PnL']) == True:
    trades_df.drop(trades_df.tail(1).index,inplace=True)

  # add to all trades
  trades_df['Ticker'] = ticker_file
  all_trades_df = pd.concat([all_trades_df, trades_df])

# save all_trades_df
os.chdir('C:/Users/Garrett/Downloads/backtest_data')
all_trades_df.to_csv(path_or_buf='all_trades_df.csv')

In [23]:
# stats
print('number of trades: ', all_trades_df.shape[0])
print('win rate: ', all_trades_df.loc[all_trades_df['PnL'] > 0].shape[0] / all_trades_df.shape[0])
print('average win :', all_trades_df.loc[all_trades_df['PnL'] > 0, 'PnL'].mean())
print('average loss :', all_trades_df.loc[all_trades_df['PnL'] < 0, 'PnL'].mean())
print('total PnL :', (all_trades_df['PnL'] + 1).prod() - 1)
print('win rate of longs: ', all_trades_df.loc[(all_trades_df['Long/Short'] == 'Long') & (all_trades_df['PnL'] > 0)].shape[0] / all_trades_df.loc[(all_trades_df['Long/Short'] == 'Long')].shape[0])
print('win rate of shorts: ', all_trades_df.loc[(all_trades_df['Long/Short'] == 'Short') & (all_trades_df['PnL'] > 0)].shape[0] / all_trades_df.loc[(all_trades_df['Long/Short'] == 'Short')].shape[0])

number of trades:  44443
win rate:  0.22232972571608578
average win : 0.04086755380573609
average loss : -0.010827632429652243
total PnL : 3.9896499328523403
win rate of longs:  0.24481892401088548
win rate of shorts:  0.1814840631138711


In [ ]:
all_trades_df['PnL'].hist()